In [5]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [6]:
# con = sqlite3.connect('../Module3/Amazon_food_review.ipynb')
con = sqlite3.connect('/Users/avneeshgautam29/Placement_learning/Machine Learning/Learning_from_Machine_Learning/Self_learning_applied_AI/Module3/amazon_dataset/database.sqlite')
filtered_data = pd.read_sql_query(""" Select * from Reviews Where Score !=3 """,con)
filtered_data.shape

(525814, 10)

In [7]:
def partition(x):
    if x>3:
        return "positive"
    else:
        return "negative"


actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition)
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(1)

Number of data points in our data (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...


In [8]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", con)

In [9]:
display.head(5)

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ESG,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B005ZBZLT4,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ESG,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBEV0,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


## Data Clearning

In [10]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [11]:
# sorting data according to Product ID
sorted_data = filtered_data.sort_values('ProductId')


# drop duplicate

final = sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Time"})
print("Original Data shape: ",sorted_data.shape)
print("shape after dropping : ", final.shape)


Original Data shape:  (525814, 10)
shape after dropping :  (328771, 10)


In [12]:
final = final[final.HelpfulnessDenominator >= final.HelpfulnessNumerator]
final.shape

(328769, 10)

In [13]:
final['Score'].value_counts()

Score
positive    275649
negative     53120
Name: count, dtype: int64

## Bag of words ( Text to vector)

In [14]:
count_vector = CountVectorizer()  #ski-learn
final_counts =  count_vector.fit_transform(final['Text'].values)
type(final_counts)

scipy.sparse._csr.csr_matrix

In [15]:
final_counts.shape

(328769, 110684)

## Text preprocessing

In [16]:
 i = 0
 for sent in final['Text'].values:
    if len(re.findall('<.?>',sent)):
        print(i)
        print(sent)
    
    i+=1

241
Michael Keaton was already on his way to being a major star when he played the ghost with the most in "Beetlejuice." His performance in the title role simply solidified his status as one of Hollywood's brightest comedy stars.<p>Now, years later, this film still generates the same belly laughs that it did when it was first released. Alec Baldwin and Geena Davis have fun in their roles as husband and wife who are surprised to find themselves dead, but they are merely window-dressing for Keaton's centerpiece. The only complaint here is that there are a few too many gross-out gags, but that is something which has come to define many of director Tim Burton's films. Still, the movie is a lot of fun from beginning to end, and it's worth considering adding it to your video collection.
250
What can I say? This is a CLASSIC! I've seen this movie so many times I actually have most of the words memorized. This movie is hilariously funny, but at the same time it has a touch of drama in it. It's

In [17]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
stop =  set(stopwords.words('english'))
sno = nltk.stem.SnowballStemmer('english')   # similar words

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/avneeshgautam29/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
def cleanhtml(scentence):
    cleanr = re.compile('<.?>')
    cleantext = re.sub(cleanr, ' ',scentence)
    return cleantext

def cleanpunc(scentence):
    cleaned= re.sub(r'[?|!|\'|"#]',r'',scentence)
    cleaned = re.sub(r'[.|,)|(|\|/)]',r' ',cleaned)
    return cleaned

print(stop)

{'himself', 'yours', "should've", 'most', 'do', 'not', 'once', 'their', 'into', "hadn't", 'yourself', 'how', "don't", 'in', 'theirs', 'off', 'myself', 'few', 'his', "weren't", 'here', 'about', 'hers', 'doesn', "you'd", 'needn', 'all', "doesn't", 'we', 'your', 'by', 'too', 'didn', "hasn't", "shouldn't", 'ours', 'won', 's', 'out', 'are', 'other', 'up', 'no', 'nor', 'with', 'same', 'a', 'those', 'weren', 'themselves', 'will', 're', 'above', 'them', 'after', 'own', 'than', 'll', 've', 'each', 'below', 'does', 'i', 'but', 'because', 'more', "aren't", 'until', 'for', 'between', 'where', "couldn't", 'from', 'aren', 'hadn', 'isn', 'me', 'y', 'shan', 'who', "won't", 'an', 'itself', 'both', 'when', 'under', 'is', 'mightn', 'which', 'over', 'what', 'she', 'these', 'against', 'why', 'very', 't', "needn't", "that'll", 'wasn', 'should', "you're", 'wouldn', 'were', 'haven', 'at', 'couldn', "didn't", 'they', 'just', 'o', 'if', 'further', 'did', "it's", 'of', 'such', 'been', 'or', "isn't", "mustn't", "

In [19]:
print(sno.stem('taste'))

tast


In [20]:
# i = 0
# str1 = ' '
# all_positive_words = []
# all_negative_words = []

# s = ''

# for sent in final['Text'].values:
#     filtered_sentence = []
#     sent = cleanhtml(sent)

#     for w in sent.split():
#         for cleaned_words in cleanpunc(w).split():
#             if( (cleaned_words.isalpha()) & (len(cleaned_words) >2)) :
#                 if(cleaned_words.lower() not in stop) :
#                     s=(sno.stem(cleaned_words.lower())).encode('utf8')
#                     filtered_sentence.append(s)
#                     if(final['Score'].values)[i] == 'positive':
#                         all_positive_words.append(s) #list of all words use
#                     if(final['Score'].values)[i] == 'negative':
#                         all_negative_words.append(s) #list of all words use
#                 else:
#                     continue
#             else:
#                 continue
# #print(filtered_sentence)
#     str1 = b" ".join(filtered_sentence) #final string of cleaned words
#     str1

In [21]:
print(all_positive_words.shape)

NameError: name 'all_positive_words' is not defined

# bi-gram

In [22]:
count_vect = CountVectorizer(ngram_range = (1,2))
final_bigram_counts = count_vect.fit_transform(final['Text'].values)


In [25]:
final_bigram_counts.shape

(328769, 2762122)

In [48]:
type(final_bigram_counts)

scipy.sparse._csr.csr_matrix

# TF-IDF

In [28]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)
final_tf_idf.shape

(328769, 2762122)

In [34]:
features = tf_idf_vect.get_feature_names_out()

In [42]:
features[100000:100010]

array(['all soapy', 'all soba', 'all soda', 'all sodas', 'all sodastream',
       'all sodium', 'all soft', 'all softened', 'all soggy',
       'all soiled'], dtype=object)

In [41]:
len(features)

2762122